## Imports

In [4]:
from bs4 import BeautifulSoup
from collections import Counter
from imblearn.over_sampling import SMOTE 
from matplotlib_venn import venn2
from scipy import stats
from sklearn import tree, svm
from sklearn.decomposition import PCA
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
# from sklearn.externals.six import StringIO 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score 
from sklearn.metrics import auc, confusion_matrix, f1_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import requests
import seaborn as sns
import warnings

pd.set_option('display.max_columns', 500)

# Leafly Scraping

In [ ]:
# webscrape 115 pages of strains from Leafly using BeautifulSoup

# create empty lists for storing the data
names,props,links=[],[],[]

# scrape data from all 71 pages of Leafly
for i in tqdm(range(1,116)):
    response=requests.get('https://www.leafly.com/strains?page={}'.format(i))
    soup=BeautifulSoup(response.content,'html.parser')
    
    # use corresponding tags for names and properties
    name=soup.findAll('div',{'class':'strain-tile__name'})
    prop=soup.findAll('span',{'class':'tag mb-md'})
    link = soup.findAll('img')
    
    for n,p,l in zip(name,prop,link):
        names.append(n.text)
        props.append(p.text)
        links.append(l.src)